In [1]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial

import yaml
import os.path as op
from tqdm import tqdm

tqdm.pandas()

### Features Detection

In [2]:
mylist = []
sg_tsv = './features_detection/training/train.feature.tsv'
for chunk in pd.read_csv(sg_tsv, sep='\t', header = None, converters={1:json.loads}, chunksize=20000):
    mylist.append(chunk)

df_feature_detection = pd.concat(mylist, axis= 0)
del mylist
df_feature_detection=df_feature_detection.rename(columns={0: "id", 1: "features"})
#df_feature_detection

In [3]:
sg_tsv = './features_detection/training/train.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection

In [4]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
28995,997338199,{'features': 's9yMPgAAAAAAAAAAWzh9P5uqOEAAAAAA...,"[{'class': 'Hair', 'conf': 0.7632176876068115,..."
28996,997722733,{'features': 'N7IGQElTrj0AAAAAxUgmPkTlSD8AAAAA...,"[{'class': 'Man', 'conf': 0.9461695551872253, ..."
28997,997876722,{'features': 'AAAAAF9gpD8AAAAAubgjPQAAAAAAAAAA...,"[{'class': 'Tomato', 'conf': 0.910000860691070..."
28998,99804383,{'features': 'iGkyPi+yeT0AAAAAMgceQHxPhDwAAAAA...,"[{'class': 'Shirt', 'conf': 0.9361402988433838..."


In [5]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

29000it [05:57, 81.13it/s] 


In [6]:
full_detection.iloc[0]["label"]

[{'class': 'Bush',
  'conf': 0.8409668207168579,
  'rect': [0.0, 216.70770263671875, 151.86502075195312, 387.5498046875],
  'feature': array([0.0293, 1.6005, 0.    , ..., 0.7751, 0.3417, 0.4561], dtype=float32),
  'bbox_id': 0},
 {'class': 'Shirt',
  'conf': 0.828758955001831,
  'rect': [175.1328887939453,
   158.813232421875,
   207.6816864013672,
   223.3404541015625],
  'feature': array([0.6272, 0.    , 0.    , ..., 0.4467, 0.1291, 0.0977], dtype=float32),
  'bbox_id': 1},
 {'class': 'Shirt',
  'conf': 0.8219393491744995,
  'rect': [206.0119171142578,
   146.23992919921875,
   257.36871337890625,
   242.8354034423828],
  'feature': array([0.0042, 0.    , 0.    , ..., 0.4857, 0.1932, 0.1542], dtype=float32),
  'bbox_id': 2},
 {'class': 'Man',
  'conf': 0.8003118634223938,
  'rect': [155.7000274658203,
   187.9111328125,
   253.99468994140625,
   374.9774475097656],
  'feature': array([7.0939, 0.    , 0.0528, ..., 0.75  , 0.3741, 0.2952], dtype=float32),
  'bbox_id': 3},
 {'class': 'S

### Features Segmentation

In [6]:
mylist = []
sg_tsv = './features_test1/training/train.feature.tsv'
for chunk in pd.read_csv(sg_tsv, sep='\t', header = None, converters={1:json.loads}, chunksize=20000):
    mylist.append(chunk)

df_feature_segmentation = pd.concat(mylist, axis= 0)
del mylist
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})
#df_feature_segmentation

In [7]:
sg_tsv = './features_test1/training/train.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation

In [8]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...,...
28995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
28996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
28997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
28998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [9]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

29000it [01:15, 382.06it/s]


In [13]:
full_segmentation.iloc[0]["label"]

[{'class': 'Belt buckle',
  'conf': 0.5350186228752136,
  'rect': [217.0223846435547,
   234.0861053466797,
   223.67523193359375,
   240.3889923095703],
  'feature': array([2.956 , 0.    , 0.    , ..., 0.4808, 0.0126, 0.02  ], dtype=float32),
  'bbox_id': 0},
 {'class': 'Tree',
  'conf': 0.8952938318252563,
  'rect': [0, 0, 332, 421],
  'feature': array([5.4538, 4.0191, 0.    , ..., 0.842 , 0.842 , 0.997 ], dtype=float32),
  'bbox_id': 1},
 {'class': 'Sky',
  'conf': 0.9932218790054321,
  'rect': [175, 0, 300, 57],
  'feature': array([ 3.0832, 22.0566,  1.2089, ...,  0.114 ,  0.114 ,  0.3754],
        dtype=float32),
  'bbox_id': 2},
 {'class': 'Person',
  'conf': 0.9997443556785583,
  'rect': [162, 112, 257, 365],
  'feature': array([0.735 , 1.7667, 5.8864, ..., 0.73  , 0.506 , 0.2853], dtype=float32),
  'bbox_id': 3},
 {'class': 'Person',
  'conf': 0.996145486831665,
  'rect': [165, 123, 214, 333],
  'feature': array([1.9702, 6.9486, 0.4648, ..., 0.666 , 0.42  , 0.1471], dtype=float

### comparison

In [101]:
full_detection.iloc[0]["label"][33]

{'class': 'Boy',
 'conf': 0.22040419280529022,
 'rect': [150.4026641845703,
  98.98582458496094,
  259.79150390625,
  382.6158447265625],
 'feature': array([4.432 , 0.2484, 0.    , ..., 0.7652, 0.5673, 0.3285], dtype=float32),
 'bbox_id': 33}

In [40]:
full_segmentation.iloc[0]["label"][3]

{'class': 'Person',
 'conf': 0.9997443556785583,
 'rect': [162, 112, 257, 365],
 'feature': array([0.735 , 1.7667, 5.8864, ..., 0.73  , 0.506 , 0.2853], dtype=float32),
 'bbox_id': 3}

In [111]:
1 - spatial.distance.cosine(full_segmentation.iloc[0]["label"][3]["feature"], full_detection.iloc[0]["label"][33]["feature"])# Boy vs Person

0.36352017521858215

In [112]:
1 - spatial.distance.cosine(full_segmentation.iloc[0]["label"][3]["rect"], full_detection.iloc[0]["label"][33]["rect"])# Boy vs Person

0.9988605741724652

In [23]:
full_detection.iloc[0]["label"][10]

{'class': 'Grass',
 'conf': 0.6013863682746887,
 'rect': [0.9849861264228821,
  357.9863586425781,
  269.23663330078125,
  486.8345031738281],
 'feature': array([0.6912, 0.    , 0.    , ..., 0.9737, 0.2577, 0.8056], dtype=float32),
 'bbox_id': 10}

In [22]:
full_segmentation.iloc[0]["label"][5]

{'class': 'Grass',
 'conf': 0.9679450988769531,
 'rect': [0, 365, 332, 499],
 'feature': array([0.3997, 8.8439, 7.3527, ..., 0.998 , 0.268 , 0.997 ], dtype=float32),
 'bbox_id': 5}

In [24]:
1 - spatial.distance.cosine(full_segmentation.iloc[0]["label"][5]["feature"], full_detection.iloc[0]["label"][10]["feature"])# Man vs Person

0.2504027187824249

In [26]:
1 - spatial.distance.cosine(full_segmentation.iloc[0]["label"][5]["rect"], full_detection.iloc[0]["label"][10]["rect"])# Man vs Person

0.9972840524412586

In [127]:
full_segmentation.iloc[0]["label"][0]

{'class': 'Belt buckle',
 'conf': 0.5350186228752136,
 'rect': [217.0223846435547,
  234.0861053466797,
  223.67523193359375,
  240.3889923095703],
 'feature': array([2.956 , 0.    , 0.    , ..., 0.4808, 0.0126, 0.02  ], dtype=float32),
 'bbox_id': 0}

In [18]:
tot_similarity = 0
count_sim = 0
for i in tqdm(range(len(full_segmentation[:100]))):
    for row_seg in full_segmentation.iloc[i]["label"]:
        for row_dect in full_detection.iloc[i]["label"]:
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.998:
                tot_similarity += 1 - spatial.distance.cosine(row_seg["feature"][:2048], row_dect["feature"][:2048])
                count_sim += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 16.49it/s]


In [19]:
tot_similarity / count_sim

0.2288026699999422

In [11]:
tot_similarity

166378.14504789747

In [12]:
count_sim

727858

In [13]:
tot_similarity / count_sim

0.22858599486149425

In [ ]:
# test con estrazione feautures direttamente dalla regione

In [14]:
import cv2, random

In [27]:
def get_size(image_size):
    min_size = 600
    max_size = 1000
    if not isinstance(min_size, (list, tuple)):
        min_size = (min_size,)
    w, h = image_size
    size = random.choice(min_size)
    if max_size is not None:
        min_original_size = float(min((w, h)))
        max_original_size = float(max((w, h)))
        if max_original_size / min_original_size * size > max_size:
            size = int(round(max_size * min_original_size / max_original_size))

    if (w <= h and w == size) or (h <= w and h == size):
        return (h, w)

    if w < h:
        ow = size
        oh = int(size * h / w)
    else:
        oh = size
        ow = int(size * w / h)

    return (oh, ow)

In [97]:
img = str(full_detection.iloc[0]["id"])+".jpg"
print("img: ", img)
im_cv2 = cv2.imread("../datasets/flickr30k_images/train/"+img)
#new_size = get_size((im_cv2.shape[1],im_cv2.shape[0]))
#im_cv2 = cv2.resize(im_cv2, (new_size[1], new_size[0]), interpolation = cv2.INTER_AREA)

x,y,w,h = tuple(full_segmentation.iloc[0]["label"][3]["rect"])
#x,y,w,h = tuple(full_detection.iloc[0]["label"][33]["rect"])

ROI = im_cv2[int(y):int(h), int(x):int(w)]
print("area: ", ROI.shape[0]*ROI.shape[1])
cv2.imshow("Man",ROI)
cv2.waitKey(0) 
cv2.destroyAllWindows() 

img:  1000092795.jpg
area:  24035


In [15]:
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.preprocessing import image

resnet = ResNet101(weights='imagenet', include_top=False, pooling='max')

In [24]:
tot_similarity_test2 = 0
count_sim_test2 = 0
for i in tqdm(range(len(full_segmentation[:100]))):
    img = str(full_detection.iloc[0]["id"])+".jpg"
    im_cv2 = cv2.imread("../datasets/flickr30k_images/train/"+img)

    for row_dect in tqdm(full_detection.iloc[i]["label"]):
        x,y,w,h = tuple(row_dect["rect"])

        ROI = im_cv2[int(y):int(h), int(x):int(w)]
        if ROI.shape[0]*ROI.shape[1] > 0:
            ROI_resized = cv2.resize(np.float32(ROI), (224, 224), interpolation = cv2.INTER_LINEAR)
            img_data = image.img_to_array(ROI_resized)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)
            feature = resnet.predict(img_data)
            feature = feature.flatten()
            row["feature_test"] = feature

 19%|██████████████████████████▏                                                                                                            | 6/31 [00:42<02:47,  6.71s/it]


 56%|███████████████████████████████████████████████████████████████████████████▏                                                          | 23/41 [03:14<02:33,  8.53s/it]


  2%|██▌                                                                                                                               | 2/100 [13:06<10:42:20, 393.27s/it]


KeyboardInterrupt: 

In [32]:
tot_similarity_test2 = 0
count_sim_test2 = 0
tot_c = 0
for i in tqdm(range(len(full_segmentation[:100]))):
    img = str(full_detection.iloc[0]["id"])+".jpg"
    im_cv2 = cv2.imread("../datasets/flickr30k_images/train/"+img)

    for row_seg in full_segmentation.iloc[i]["label"]:
        for row_dect in full_detection.iloc[i]["label"]:
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.998:
                x,y,w,h = tuple(row_dect["rect"])

                ROI = im_cv2[int(y):int(h), int(x):int(w)]
                if ROI.shape[0]*ROI.shape[1] > 0:
                    #ROI_resized = cv2.resize(np.float32(ROI), (224, 224), interpolation = cv2.INTER_LINEAR)
                    #img_data = image.img_to_array(ROI_resized)
                    #img_data = np.expand_dims(img_data, axis=0)
                    #img_data = preprocess_input(img_data)
                    #feature = resnet.predict(img_data)
                    #feature = feature.flatten()
                    #tot_similarity_test2 += 1 - spatial.distance.cosine(row_seg["feature"][:2048], feature)
                    count_sim_test2 += 1
                tot_c += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 18.55it/s]


In [81]:
tot_similarity_test2 = 0
count_sim_test2 = 0
tot_c = 0
full_not_present = []
for i in tqdm(range(len(full_detection))):    
    not_present = []
    for row_dect in full_detection.iloc[i]["label"]:
        check = False
        for row_seg in full_segmentation.iloc[i]["label"]:
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.985:
                check = True
                break
        if not check:
            not_present.append(row_dect)
    full_not_present.append(not_present)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 35.15it/s]


In [50]:
len(full_detection.iloc[0]["label"])

37

In [84]:
full_segmentation.iloc[0]["label"]

[{'class': 'Belt buckle',
  'conf': 0.5350186228752136,
  'rect': [217.0223846435547,
   234.0861053466797,
   223.67523193359375,
   240.3889923095703],
  'feature': array([2.956 , 0.    , 0.    , ..., 0.4808, 0.0126, 0.02  ], dtype=float32),
  'bbox_id': 0},
 {'class': 'Tree',
  'conf': 0.8952938318252563,
  'rect': [0, 0, 332, 421],
  'feature': array([5.4538, 4.0191, 0.    , ..., 0.842 , 0.842 , 0.997 ], dtype=float32),
  'bbox_id': 1},
 {'class': 'Sky',
  'conf': 0.9932218790054321,
  'rect': [175, 0, 300, 57],
  'feature': array([ 3.0832, 22.0566,  1.2089, ...,  0.114 ,  0.114 ,  0.3754],
        dtype=float32),
  'bbox_id': 2},
 {'class': 'Person',
  'conf': 0.9997443556785583,
  'rect': [162, 112, 257, 365],
  'feature': array([0.735 , 1.7667, 5.8864, ..., 0.73  , 0.506 , 0.2853], dtype=float32),
  'bbox_id': 3},
 {'class': 'Person',
  'conf': 0.996145486831665,
  'rect': [165, 123, 214, 333],
  'feature': array([1.9702, 6.9486, 0.4648, ..., 0.666 , 0.42  , 0.1471], dtype=float

In [82]:
len(full_not_present[0])

18

In [83]:
full_not_present[0]

[{'class': 'Bush',
  'conf': 0.8409668207168579,
  'rect': [0.0, 216.70770263671875, 151.86502075195312, 387.5498046875],
  'feature': array([0.0293, 1.6005, 0.    , ..., 0.7751, 0.3417, 0.4561], dtype=float32),
  'bbox_id': 0,
  'feature_estratte': array([5.3435, 0.    , 0.    , ..., 0.5898, 0.3866, 0.3831], dtype=float32)},
 {'class': 'Shirt',
  'conf': 0.8219393491744995,
  'rect': [206.0119171142578,
   146.23992919921875,
   257.36871337890625,
   242.8354034423828],
  'feature': array([0.0042, 0.    , 0.    , ..., 0.4857, 0.1932, 0.1542], dtype=float32),
  'bbox_id': 2,
  'feature_estratte': array([ 2.5183, 10.739 ,  1.0527, ...,  2.6162,  7.1781,  4.8053],
        dtype=float32)},
 {'class': 'Hair',
  'conf': 0.7583618760108948,
  'rect': [197.89041137695312,
   111.39276123046875,
   236.45010375976562,
   145.28573608398438],
  'feature': array([0.074 , 0.    , 0.    , ..., 0.2906, 0.0678, 0.1158], dtype=float32),
  'bbox_id': 5,
  'feature_estratte': array([15.2238,  5.9507, 

In [37]:
full_detection

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
28995,997338199,{'features': 's9yMPgAAAAAAAAAAWzh9P5uqOEAAAAAA...,"[{'class': 'Hair', 'conf': 0.7632176876068115,..."
28996,997722733,{'features': 'N7IGQElTrj0AAAAAxUgmPkTlSD8AAAAA...,"[{'class': 'Man', 'conf': 0.9461695551872253, ..."
28997,997876722,{'features': 'AAAAAF9gpD8AAAAAubgjPQAAAAAAAAAA...,"[{'class': 'Tomato', 'conf': 0.910000860691070..."
28998,99804383,{'features': 'iGkyPi+yeT0AAAAAMgceQHxPhDwAAAAA...,"[{'class': 'Shirt', 'conf': 0.9361402988433838..."


In [35]:
full_segmentation.iloc[0]["label"]

[{'class': 'Belt buckle',
  'conf': 0.5350186228752136,
  'rect': [217.0223846435547,
   234.0861053466797,
   223.67523193359375,
   240.3889923095703],
  'feature': array([2.956 , 0.    , 0.    , ..., 0.4808, 0.0126, 0.02  ], dtype=float32),
  'bbox_id': 0},
 {'class': 'Tree',
  'conf': 0.8952938318252563,
  'rect': [0, 0, 332, 421],
  'feature': array([5.4538, 4.0191, 0.    , ..., 0.842 , 0.842 , 0.997 ], dtype=float32),
  'bbox_id': 1},
 {'class': 'Sky',
  'conf': 0.9932218790054321,
  'rect': [175, 0, 300, 57],
  'feature': array([ 3.0832, 22.0566,  1.2089, ...,  0.114 ,  0.114 ,  0.3754],
        dtype=float32),
  'bbox_id': 2},
 {'class': 'Person',
  'conf': 0.9997443556785583,
  'rect': [162, 112, 257, 365],
  'feature': array([0.735 , 1.7667, 5.8864, ..., 0.73  , 0.506 , 0.2853], dtype=float32),
  'bbox_id': 3},
 {'class': 'Person',
  'conf': 0.996145486831665,
  'rect': [165, 123, 214, 333],
  'feature': array([1.9702, 6.9486, 0.4648, ..., 0.666 , 0.42  , 0.1471], dtype=float

In [26]:
tot_similarity_test2

0

In [33]:
count_sim_test2

1594

In [34]:
tot_c

1594

In [ ]:
tot_similarity_test2 / count_sim_test2

In [ ]:
ROI_resized = cv2.resize(np.float32(ROI), (224, 224), interpolation = cv2.INTER_LINEAR)
img_data = image.img_to_array(ROI_resized)
img_data = np.expand_dims(img_data, axis=0)
img_data = preprocess_input(img_data)
feature = resnet.predict(img_data)
feature = feature.flatten()
len(feature)

In [100]:
1 - spatial.distance.cosine(full_detection.iloc[0]["label"][3]["feature"][:2048], feature) # segmentation

0.320299357175827

In [96]:
1 - spatial.distance.cosine(full_detection.iloc[0]["label"][3]["feature"][:2048], feature) # detection

0.3323040008544922